# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la palma,33.8464,-118.0467,18.73,69,100,1.54,US,1670015459
1,1,avarua,-21.2078,-159.7750,28.03,78,40,4.12,CK,1670015460
2,2,vaitape,-16.5167,-151.7500,26.86,77,43,8.05,PF,1670015460
3,3,tanabe,33.7333,135.3667,5.95,71,0,1.65,JP,1670015461
4,4,pangnirtung,66.1451,-65.7125,-8.00,86,100,6.17,CA,1670015461


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
url = f"https://maps.geoapify.com/v1/styles/osm-carto/style.json?apiKey={geoapify_key}"
response = requests.get(url).json()
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
weather_condition_df = city_data_df
weather_condition_df = weather_condition_df.loc[(weather_condition_df["Max Temp"] < 27) & (weather_condition_df["Max Temp"] > 21)]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Wind Speed"] < 4.5]
weather_condition_df = weather_condition_df.loc[weather_condition_df["Cloudiness"] == 0]

# Drop any rows with null values
weather_condition_df = weather_condition_df.dropna()

# Display sample data
weather_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
215,215,mar del plata,-38.0023,-57.5575,22.13,68,0,3.13,AR,1670015343
304,304,gwadar,25.1216,62.3254,23.90,40,0,2.59,PK,1670015636
321,321,gawler,-34.6000,138.7333,23.04,49,0,2.49,AU,1670015645
332,332,port augusta,-32.5000,137.7667,23.82,26,0,1.62,AU,1670015652
352,352,adrar,20.5022,-10.0711,21.64,17,0,3.77,MR,1670015664
391,391,barentu,15.1139,37.5928,21.82,53,0,2.21,ER,1670015686
506,506,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1670015752
586,586,tombouctou,20.0000,-3.0000,21.64,17,0,0.74,ML,1670015799


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_condition_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel name"] = pd.Series(dtype="str")

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
215,215,mar del plata,-38.0023,-57.5575,22.13,68,0,3.13,AR,1670015343,NaN
304,304,gwadar,25.1216,62.3254,23.90,40,0,2.59,PK,1670015636,NaN
321,321,gawler,-34.6000,138.7333,23.04,49,0,2.49,AU,1670015645,NaN
332,332,port augusta,-32.5000,137.7667,23.82,26,0,1.62,AU,1670015652,NaN
352,352,adrar,20.5022,-10.0711,21.64,17,0,3.77,MR,1670015664,NaN
391,391,barentu,15.1139,37.5928,21.82,53,0,2.21,ER,1670015686,NaN
506,506,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1670015752,NaN
586,586,tombouctou,20.0000,-3.0000,21.64,17,0,0.74,ML,1670015799,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = -38.0023,-57.5575,25.1216,62.3254
    params["bias"] = -34.6000,138.7333
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mar del plata - nearest hotel: No hotel found
gwadar - nearest hotel: No hotel found
gawler - nearest hotel: No hotel found
port augusta - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
barentu - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
tombouctou - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel Name
215,215,mar del plata,-38.0023,-57.5575,22.13,68,0,3.13,AR,1670015343,NaN,No hotel found
304,304,gwadar,25.1216,62.3254,23.90,40,0,2.59,PK,1670015636,NaN,No hotel found
321,321,gawler,-34.6000,138.7333,23.04,49,0,2.49,AU,1670015645,NaN,No hotel found
332,332,port augusta,-32.5000,137.7667,23.82,26,0,1.62,AU,1670015652,NaN,No hotel found
352,352,adrar,20.5022,-10.0711,21.64,17,0,3.77,MR,1670015664,NaN,No hotel found
391,391,barentu,15.1139,37.5928,21.82,53,0,2.21,ER,1670015686,NaN,No hotel found
506,506,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1670015752,NaN,No hotel found
586,586,tombouctou,20.0000,-3.0000,21.64,17,0,0.74,ML,1670015799,NaN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
url = f"https://api.geoapify.com/v2/placesapiKey={geoapify_key}"
response = requests.get(url).json()
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]